# **<p style="text-align: center;">Aprendizaje por refuerzo - Ingeniería en Inteligencia Artificial</p>**
## **<p style="text-align: center;">Trabajo práctico Final - SAC</p>**

#### <p style="text-align: center;">Pettinari Fausto, Schuemer Ignacio, Torres Santiago </p>
#### <p style="text-align: center;">Profesores: Claudio Pose, Gabriel Torre, Nicolás Romero, Tomás Chimenti</p>

# **Bipedal Walker Enviroment**

In [1]:
import sys
sys.path.append('..')

In [2]:
import gymnasium as gym
from gymnasium.wrappers import TimeLimit
from sac.utils.logger_utils import *

In [3]:
env = gym.make("BipedalWalker-v3")
env = TimeLimit(env, max_episode_steps=1600)

#### **Logging & Debugging (Random Agent)**

In [4]:
from torch.utils.tensorboard import SummaryWriter
from sac.random_agent import random_agent_loop

2025-11-30 13:02:47.888079: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-30 13:02:47.934379: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-30 13:02:48.990888: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [5]:
seed = 42
# writer = SummaryWriter(f"../runs/BipedalWalker-v3/")
num_episodes = 500
# random_agent_loop(env, num_episodes, writer, seed)

## **SAC Agent**

In [6]:
import yaml
from sac.agent import SAC

In [7]:
with open('configs/bipedal_walker.yaml', 'r') as f:
    config = yaml.safe_load(f)

### **TRAIN**

In [8]:
SAC_agent = SAC(env=env, config=config)
SAC_agent.run_training_loop(num_episodes=num_episodes, tqdm_disable=False, print_rewards=False)

100%|██████████| 500/500 [1:35:49<00:00, 11.50s/it]

Agent saved to runs/BipedalWalker-v3/SAC/sac-bipedal-walker-2025_11_30-13_02_50/sac_agent.pth


{'total_episodes': 500,
 'best_avg_return': np.float32(256.5958),
 'final_avg_return': np.float32(255.32434)}

In [9]:
num_episodes = 500
config['sac']['auto_entropy_tuning'] = True
config['sac']['alpha_lr'] = 0.0001
config['logger']['agent_name'] = 'sac-dynamic-lr_0.0001'
SAC_agent = SAC(env=env, config=config)
SAC_agent.run_training_loop(num_episodes=num_episodes, tqdm_disable=False, print_rewards=False)

100%|██████████| 500/500 [48:53<00:00,  5.87s/it]  

Agent saved to runs/BipedalWalker-v3/sac-dynamic-lr_0.0001/sac-bipedal-walker-2025_11_30-14_38_40/sac_agent.pth


{'total_episodes': 500,
 'best_avg_return': np.float32(66.98057),
 'final_avg_return': np.float32(63.958485)}

#### **Eval trained Agent**

In [10]:
# SAC_agent.eval_agent(num_episodes=1, render_mode="human")

### **Load trained agent & Eval**

In [11]:
# # Load trained agent and run in environment
# config['logger']['enabled'] = False  # Disable logging for loaded agent
# SAC_agent_loaded = SAC(env=env, config=config)
# SAC_agent_loaded.load_agent('../notebooks/runs/BipedalWalker-v3/SAC/sac-bipedal-walker-2025_11_25-16_41_19/sac_agent.pth')

In [12]:
# SAC_agent_loaded.eval_agent(num_episodes=1, render_mode="human")

## **SB3 SAC Agent**

In [13]:
# from stable_baselines3 import SAC as SB3_SAC
# from sac.utils.stable_baseline_params import get_sb3_sac_params
# from sac.utils.stable_baseline_logger import RobustEpisodeLogger

# with open('configs/bipedal_walker_sb3.yaml', 'r') as f:
#     config = yaml.safe_load(f)

# sb3_params = get_sb3_sac_params(env, config, seed, env_id="BipedalWalker-v3")

# keys_to_keep = ["policy", "env"]
# sb3_params = {k: v for k, v in sb3_params.items() if k in keys_to_keep}

# writer = SummaryWriter(f"runs/BipedalWalker-v3/sac_sb3-2/")
# sb3_sac = SB3_SAC(**sb3_params)
# callback = RobustEpisodeLogger(
#     writer=writer,
#     max_episodes=num_episodes,
#     save_dir="runs/BipedalWalker-v3/sac_sb3-2/",
#     verbose=0
# )

# sb3_sac.learn(
#     total_timesteps=num_episodes * env.spec.max_episode_steps,
#     callback=callback,
#     progress_bar=True
# )
# # save the trained agent
# sb3_sac.save("runs/BipedalWalker-v3/sac_sb3-2/sb3_sac_agent")

# writer.close()
# env.close()